<a href="https://colab.research.google.com/github/andrevictorm/PROJETOS/blob/main/Mini_Projeto_Final_Nivel_Parte2_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalando Pyspark
 #crtl f9
pip install pyspark


SyntaxError: ignored

In [ ]:
#Importando funcções padrão. Função Window também importada porque será usado mais na frente
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,BooleanType
import pandas as pd


In [ ]:
#Montando estrutura
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [ ]:
#importando dataframe
df = (spark
       .read
       .format("csv")
       .option("header", True)
       .option("inferschema", True)
       .option("delimiter", ",")
       .load('/content/drive/MyDrive/Colab Notebooks/markting_campeditado.csv')
)

In [ ]:
#Exibindo dataframe
df.show()
df.printSchema()

+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+-------------+---------+--------+
|  ID|ano_nascimento| Educaçao|estado_civil|  renda|crianca_casa|jovem_casa|Dt_cliente|recencia|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|NumVisitaWebMes|Z_CostContact|Z_Revenue|Response|
+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+-------------+---------+--------+
|5524|          1957|Graduacao|    Solteiro|58138.0|           0|         0|04-09-2012|      58|     635|      88|     546|     172|     88|     88|                3|           8|               10|            4|      

In [ ]:
#Alterando nomes de coluna
df = df.withColumnRenamed("Z_Revenue", "Z_Receita").withColumnRenamed("Z_CostContact", "Z_Custocontato").withColumnRenamed("Response", "Resposta").withColumnRenamed("Educaçao", "Formacao")

In [ ]:
df.show()

+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|  ID|ano_nascimento| Formacao|estado_civil|  renda|crianca_casa|jovem_casa|Dt_cliente|recencia|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|NumVisitaWebMes|Z_Custocontato|Z_Receita|Resposta|Soma_Total_Mnt|Total_Compra_Realizada|
+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|5524|          1957|Graduacao|    Solteiro|58138.0|           0|         0|04-09-2012|      58|    

In [ ]:
#Criando uma nova coluna que soma todos os valores de Mnt
df = df.withColumn("Soma_Total_Mnt", F.col("MntVinho") + F.col("MntFruta")+F.col("MntCarne") + F.col("MntPeixe")+F.col("MntDoce") + F.col("MntOuro"))

In [ ]:
#Criando uma nova coluna que soma todos os valores de Compra realizada
df = df.withColumn("Total_Compra_Realizada", F.col("NumCompraPromocao") + F.col("NumCompraCatalogo")+F.col("NumCompraWeb") + F.col("NumCompraLoja"))

In [ ]:
df.show()

+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|  ID|ano_nascimento| Educaçao|estado_civil|  renda|crianca_casa|jovem_casa|Dt_cliente|recencia|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|NumVisitaWebMes|Z_Custocontato|Z_Receita|Resposta|Soma_Total_Mnt|Total_Compra_Realizada|
+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|5524|          1957|Graduacao|    Solteiro|58138.0|           0|         0|04-09-2012|      58|    

In [ ]:
#Agrupando a soma total do Mnt dividada por nivel de escolaridade
df.groupBy(F.col("Educaçao")).sum("Soma_Total_Mnt").show()

+---------+-------------------+
| Educaçao|sum(Soma_Total_Mnt)|
+---------+-------------------+
|Graduacao|             698626|
|      PhD|             326791|
|   Mestre|             226359|
|   Basico|               4417|
|   2º ano|             100795|
+---------+-------------------+



In [ ]:
#Rank do cliente que teve o menor Mnt para o maior
w0 = Window.partitionBy(F.col("Educaçao")).orderBy("Soma_Total_Mnt")
df.withColumn("Rank_Total_Mnt",F.dense_rank().over(w0)).select("ID","Educaçao","renda","Dt_cliente","MntVinho","MntFruta","MntCarne",'MntPeixe',"MntDoce",'MntOuro',"Soma_Total_Mnt",'Rank_Total_Mnt').show()

+-----+--------+-------+----------+--------+--------+--------+--------+-------+-------+--------------+--------------+
|   ID|Educaçao|  renda|Dt_cliente|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|Soma_Total_Mnt|Rank_Total_Mnt|
+-----+--------+-------+----------+--------+--------+--------+--------+-------+-------+--------------+--------------+
|10067|  2º ano|25176.0|10-08-2013|       4|       0|       4|       0|      0|      5|            13|             1|
|10241|  2º ano|11448.0|15-12-2013|       0|       0|       0|       6|      2|      6|            14|             2|
| 7264|  2º ano|52195.0|12-05-2014|      12|       0|       4|       0|      0|      1|            17|             3|
|10401|  2º ano|26326.0|04-09-2013|       1|       0|       5|       0|      3|      9|            18|             4|
| 9850|  2º ano|24884.0|29-11-2012|       3|       6|       7|       0|      0|      3|            19|             5|
| 4345|  2º ano|   null|12-01-2014|       5|       1|   

In [ ]:
#Agrupando a soma total de compras dividada por nivel de escolaridade
df.groupBy(F.col("Educaçao")).sum("Total_Compra_Realizada").show()

+---------+---------------------------+
| Educaçao|sum(Total_Compra_Realizada)|
+---------+---------------------------+
|Graduacao|                      16894|
|      PhD|                       7693|
|   Mestre|                       5523|
|   Basico|                        379|
|   2º ano|                       2802|
+---------+---------------------------+



In [ ]:
#Agrupador por esolaridade, 
(df.groupBy(F.col("Educaçao")).agg(
            F.sum("NumCompraPromocao").alias("Total_CompraPromocao"),
            F.sum("NumCompraWeb").alias("Total_CompraWeb"),
            F.sum("NumCompraCatalogo").alias("Total_CompraCatalogo"),
            F.sum("NumCompraLoja").alias("Total_CompraLoja")
            ).show()
)



+---------+--------------------+---------------+--------------------+----------------+
| Educaçao|Total_CompraPromocao|Total_CompraWeb|Total_CompraCatalogo|Total_CompraLoja|
+---------+--------------------+---------------+--------------------+----------------+
|Graduacao|                2603|           4649|                3072|            6570|
|      PhD|                1154|           2150|                1443|            2946|
|   Mestre|                 898|           1492|                 951|            2182|
|   Basico|                  97|            102|                  26|             154|
|   2º ano|                 456|            757|                 471|            1118|
+---------+--------------------+---------------+--------------------+----------------+



In [ ]:
#Quantidade de pessoas por escolaridade
df.groupBy(F.col("Educaçao")).count().show()


+---------+-----+
| Educaçao|count|
+---------+-----+
|Graduacao| 1127|
|      PhD|  486|
|   Mestre|  370|
|   Basico|   54|
|   2º ano|  203|
+---------+-----+



In [ ]:
#Rank do cliente que teve a menor qnt de Compra para o maior
w1 = Window.partitionBy(F.col("Educaçao")).orderBy("Total_Compra_Realizada")
df.withColumn("Rank_Total_Compra",F.dense_rank().over(w1)).select("ID","Educaçao","renda","Dt_cliente","NumCompraPromocao","NumCompraWeb","NumCompraCatalogo",'NumCompraLoja',"Total_Compra_Realizada",'Rank_Total_Compra').show()


+-----+--------+-------+----------+-----------------+------------+-----------------+-------------+----------------------+-----------------+
|   ID|Educaçao|  renda|Dt_cliente|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|Total_Compra_Realizada|Rank_Total_Compra|
+-----+--------+-------+----------+-----------------+------------+-----------------+-------------+----------------------+-----------------+
| 8720|  2º ano|   null|12-08-2012|                0|           0|                0|            1|                     1|                1|
|10401|  2º ano|26326.0|04-09-2013|                1|           0|                0|            3|                     4|                2|
| 7264|  2º ano|52195.0|12-05-2014|                1|           1|                0|            2|                     4|                2|
|11004|  2º ano|60182.0|17-05-2014|                1|           1|                0|            2|                     4|                2|
| 2166|  2º ano|4677

In [ ]:
#Calcular a media de quantidade de compra feita, media de Mnt

(df.withColumn("row",F.row_number().over(w0))
  .withColumn("Media_Compra_Qnt",F.avg(F.col("Total_Compra_Realizada")).over(w0))
  .withColumn("Media_Mnt_Total",F.avg(F.col("Soma_Total_Mnt")).over(w0))
  .select("row","ID","Educaçao","Media_Compra_Qnt","Media_Mnt_Total").show()
)


+---+-----+--------+------------------+------------------+
|row|   ID|Educaçao|  Media_Compra_Qnt|   Media_Mnt_Total|
+---+-----+--------+------------------+------------------+
|  1|10067|  2º ano|               4.0|              13.0|
|  2|10241|  2º ano|               4.0|              13.5|
|  3| 7264|  2º ano|               4.0|14.666666666666666|
|  4|10401|  2º ano|               4.0|              15.5|
|  5| 9850|  2º ano|               4.0|              16.2|
|  6| 4345|  2º ano|               4.0|              17.0|
|  7|11004|  2º ano| 4.444444444444445|18.666666666666668|
|  8|  577|  2º ano| 4.444444444444445|18.666666666666668|
|  9| 8702|  2º ano| 4.444444444444445|18.666666666666668|
| 10| 2166|  2º ano| 4.333333333333333|             19.75|
| 11| 2217|  2º ano| 4.333333333333333|             19.75|
| 12| 4102|  2º ano| 4.333333333333333|             19.75|
| 13| 1890|  2º ano|4.3076923076923075|20.153846153846153|
| 14| 5680|  2º ano|             4.375|              22.

In [ ]:
#Filtragem de quem realizou mais de 20 compras e tem Mnt maior que 1500
df.filter('Total_Compra_Realizada > 20 and Soma_Total_Mnt > 1500 ').show()


+----+--------------+---------+------------+--------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|  ID|ano_nascimento| Educaçao|estado_civil|   renda|crianca_casa|jovem_casa|Dt_cliente|recencia|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|NumVisitaWebMes|Z_Custocontato|Z_Receita|Resposta|Soma_Total_Mnt|Total_Compra_Realizada|
+----+--------------+---------+------------+--------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|5524|          1957|Graduacao|    Solteiro| 58138.0|           0|         0|04-09-2012|      58|

In [ ]:
#Pessoas maiores de 18 anos
df.filter('(2021 - ano_nascimento) > 18').show()


+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|  ID|ano_nascimento| Educaçao|estado_civil|  renda|crianca_casa|jovem_casa|Dt_cliente|recencia|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|NumVisitaWebMes|Z_Custocontato|Z_Receita|Resposta|Soma_Total_Mnt|Total_Compra_Realizada|
+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|5524|          1957|Graduacao|    Solteiro|58138.0|           0|         0|04-09-2012|      58|    

In [ ]:
#Visualização atual
df.show()

+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|  ID|ano_nascimento| Educaçao|estado_civil|  renda|crianca_casa|jovem_casa|Dt_cliente|recencia|MntVinho|MntFruta|MntCarne|MntPeixe|MntDoce|MntOuro|NumCompraPromocao|NumCompraWeb|NumCompraCatalogo|NumCompraLoja|NumVisitaWebMes|Z_Custocontato|Z_Receita|Resposta|Soma_Total_Mnt|Total_Compra_Realizada|
+----+--------------+---------+------------+-------+------------+----------+----------+--------+--------+--------+--------+--------+-------+-------+-----------------+------------+-----------------+-------------+---------------+--------------+---------+--------+--------------+----------------------+
|5524|          1957|Graduacao|    Solteiro|58138.0|           0|         0|04-09-2012|      58|    

In [ ]:
#Salvando dataframe no Google Drive
df.toPandas().to_csv('/content/drive/MyDrive/Colab Notebooks/markting_campedpyspark.csv', index = None, header=True)

